In [1]:
# Load Libraries
import pandas as pd
import numpy as np

# Set random seed
np.random.seed(42)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Load Data

In [2]:
# Load data to a DataFrame
beanDF = pd.read_excel('DryBeanDataset/Dry_Bean_Dataset.xlsx')

In [3]:
# Get a look at the data
beanDF.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [4]:
# Look at the shape of the DataFrame
beanDF.shape

(13611, 17)

In [5]:
# See the relative numbers of each target category
beanDF.value_counts('Class')

Class
DERMASON    3546
SIRA        2636
SEKER       2027
HOROZ       1928
CALI        1630
BARBUNYA    1322
BOMBAY       522
dtype: int64

## Split Training and Testing Data

In [6]:
# Load libraries
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
# Set up features target sets
X = beanDF[beanDF.columns[:-1]]
y = beanDF.Class

Let's try this without encoding the target:

In [8]:
# # Encode the target variables
# le = LabelEncoder()
# y = le.fit_transform(y)

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, 
                                                    random_state=42)

In [10]:
# Standardize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test  = scaler.transform(X_test)

# Modeling

Used GridSearchCV to find the best parameters for each modeling type

In [39]:
models_dict = {}

In [40]:
for k in models_dict:
#     print(v.score(X_test, y_test))
    print(k)
    print(models_dict[k])

## Logistic Regression

In [41]:
# Load libraries
from sklearn.linear_model import LogisticRegression

In [42]:
# Create logistic regression
logistic = LogisticRegression(C=100000, 
                              multi_class='ovr', 
                              penalty='l2', 
                              solver='liblinear', 
                              class_weight='balanced', 
                              random_state=42, 
                              max_iter=200)

In [43]:
%%time
models_dict['Logistic'] = logistic.fit(X_train, y_train)

Wall time: 995 ms


## Random Forest

In [44]:
# Load libraries
from sklearn.ensemble import RandomForestClassifier

In [46]:
# Create classifier
rfclassifier = RandomForestClassifier(criterion='entropy', 
                                      max_features='sqrt', 
                                      n_estimators=100, 
                                      random_state=42,
                                      class_weight='balanced', 
                                      n_jobs=-1)

In [47]:
%%time
models_dict['RandomForest'] = rfclassifier.fit(X_train, y_train)

Wall time: 1.07 s


## Decision Tree

In [50]:
# Load libraries
from sklearn.tree import DecisionTreeClassifier

In [51]:
# Create decision tree regressor object
decisiontree = DecisionTreeClassifier(criterion='entropy', 
                                      max_features='sqrt', 
                                      splitter='best', 
                                      random_state=42, 
                                      class_weight='balanced')

In [52]:
%%time
models_dict['DecisionTree'] = decisiontree.fit(X_train, y_train)

Wall time: 101 ms


## AdaBoost

In [53]:
# Load libraries
from sklearn.ensemble import AdaBoostClassifier

In [54]:
# Create ...
adaboost = AdaBoostClassifier(algorithm='SAMME', 
                              n_estimators=100, 
                              random_state=42)

In [55]:
%%time
models_dict['AdaBoost'] = adaboost.fit(X_train, y_train)

Wall time: 2.54 s


## Support Vector Classifier (Linear)

In [56]:
# Load libraries
from sklearn.svm import LinearSVC

In [59]:
# Create support vector classifier
linSVC = LinearSVC(C=178, 
                   loss='hinge', 
                   multi_class='crammer_singer', 
                   penalty='l1', 
                   random_state=42, 
                   dual=False, 
                   class_weight='balanced', 
                   max_iter=1000000)

In [60]:
%%time
models_dict['LinearSVC'] = linSVC.fit(X_train, y_train)

Wall time: 5.96 s


## Support Vector Classifier (SVC)

In [61]:
# Load libraries
from sklearn.svm import SVC

In [62]:
# Create support vector classifier
svc = SVC(C=32, 
          decision_function_shape='ovo', 
          gamma='scale', 
          kernel='rbf', 
          shrinking=True, 
          random_state=42, 
          class_weight='balanced', 
          max_iter=5000)

In [64]:
%%time
models_dict['SVC'] = svc.fit(X_train, y_train)

Wall time: 629 ms


In [74]:
for key in models_dict:
    model = models_dict[key]
    score = model.score(X_test, y_test)*100
    print(f"{key}:")
#     print(f"  {model}")
    print(f"  Accuracy: {round(score, 2)} %\n")
    
del(model, score)

Logistic:
  Accuracy: 92.36 %

RandomForest:
  Accuracy: 92.41 %

DecisionTree:
  Accuracy: 88.34 %

AdaBoost:
  Accuracy: 79.29 %

LinearSVC:
  Accuracy: 92.46 %

SVC:
  Accuracy: 93.22 %

